In [1]:
import math
import io

#파일 압축 용도
import gzip
import pickle
import zlib

# 데이터, 배열
import pandas as pd
import numpy as np

# 범주형 수치형 변환
from sklearn.preprocessing import LabelEncoder

np.random.seed(2016)
transformers={}

In [2]:
# 라벨 인코더 함수
def label_encoder(df, features, name):
    # 데이터 프레임 df의 변수 name값을 모두 string으로 변환
    df[name] = df[name].astype('str')
    
    #이미 라벨 인코더 했던 변수는 trasformer[name]에 있는 라벨 인코더를 재활용
    if name in transformers:
        df[name] = transformers[name].transform(df[name])
        
    #처음 나오는 변수는 transformer에 라벨인코더를 저장하고 fit_transfrom으로 라벨인코딩
    else:
        transformers[name] = LabelEncoder()
        df[name] = transformers[name].fit_transform(df[name])
        # 라벨인코딩한 변수는 features 리스트에 추가
        features.append(name)

In [ ]:
# 자체 구현 one hot encoder
def custom_one_hot(df, features, name, names, dtype = np.int8, check = False):
    

In [3]:
# 빈도 상위 100개 데이터 순위 변수 추출
def encode_top(s, count=100, dtype = np.int8):
    # 고유값 빈도 계산
    uniqs, freqs = np.unique(s, return_counts=True)
    # 빈도 top 100 추출
    top = sorted(zip(uniqs, freqs), key = lambda vk : vk[1], reverse= True)[:count]
    
    # 기존데이터 : 순위 dict 생성
    top_map = {uf[0] : l + 1 for uf, l in zip(top, range(len(top)))}
    
    # 고빈도 100개의 데이터는 순위로 대체, 그외는 0으로 대체
    return s.map(lambda x: top_map.get(x,0)).astype(dtype)

In [4]:
#날짜데이터를 월 단위 숫자로 변환 utils.py

def date_to_float(str_date):
    if str_date.__class__ is float and math.isnan(str_date) or str_date =="":
        return np.nan

    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    float_date = float(Y) * 12 + float(M)
    return float_date


#날짜데이터를 월단위로 변환하여 1~18사이로 제한
def date_to_int(str_date):
    Y,M,D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    assert 1 <= int_date <= 12+6
    return int_date
